In [1]:
import os
from dotenv import load_dotenv

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- Import the Plugin

print("✅ ADK components imported successfully.")

load_dotenv()

✅ ADK components imported successfully.


True

In [ ]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"] 
# print(GOOGLE_API_KEY)

In [3]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

✅ Callback created.


In [25]:
# get list of mcp tools from the given mcp server.. in this case it is custom mcp server to access users' GMAIL
from google.adk.tools.mcp_tool.mcp_toolset import (
    McpToolset
)
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams

async def get_tools_async():
    """Gets tools from the File System MCP Server."""
    tools = McpToolset(
        connection_params=SseServerParams(
            url="http://localhost:8001/sse",
        )
    )
    print("MCP Toolset created successfully.")
    return tools

In [22]:
dir(McpToolset)

['__abstractmethods__',
 '__annotate_func__',
 '__annotations_cache__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_tool_selected',
 'close',
 'from_config',
 'get_tools',
 'get_tools_with_prefix',
 'process_llm_request']

In [45]:
# Define constants used throughout the notebook
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"


# Google Search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="""Use the google_search tool to find information on the given topic. Return the search results.""",
    tools=[google_search]
)

mcp_tools = await get_tools_async()
print(f"Fetched tools from MCP server.")

# Agent with GMAIL access
# Google Search agent
gmail_search_agent = LlmAgent(
    name="gmail_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information in the user's gmail using given MCP toolset.",
    instruction="""Use the `get_gmail_labels` tool to get list of available labels in the user's gamil account. Please give as input users' email address to this tool.
    If you don't have email address please ask the user to give their email address.
    If you receive a exception when calling the `get_gmail_tools` with this string in the exception message `User not authenticated.. Please authenticate again...`, 
    then first authenticate the user by calling the tool `authenticate` which should return `user_id` that is the users' email address.

    You can then again try to call `get_gmail_labels` tool with the returned `user_id` from above step. And return the result from `get_gmail_labels` tool
    """,
    tools=[mcp_tools]
)

# Agent with automatic memory saving
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="HelpingAgent",
    instruction="""You are a Helper agent for the user. User can give you some information like what job he applied at what time. 
    You need to remember that and collate and help with all the past details when user wants to recollect.

    You also have "google_search_tool" which you can use to get details on the given topic asked by user like if user asks: 
    "Summarize details on XYZ company". You can use LinkedIn or any other web page to get details on XYZ company and summarize it and present it to the user.

    You have "gmail_search_agent" as a tool to get details from user's gmail account. You can use this to get requested details from user's gmail account like:
    - list all labels from the user's gmail  using the tool `get_gmail_labels`, 
    - read messages in the users' gmail account using the tool `get emails`.
    Display the output to the user in readble and understandable format. 

    Output to the user should be in a readable format, with the details of the website where you found the details.. If the details are from the memory 
    then mention accordingly. And if the details are from GMAIL mention accordingly.
    """,
    tools=[AgentTool(agent=google_search_agent), load_memory, AgentTool(agent=gmail_search_agent)],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)



print("✅ Agent created with automatic memory saving!")

print("✅ Agent created")

MCP Toolset created successfully.
Fetched tools from MCP server.
✅ Agent created with automatic memory saving!
✅ Agent created


In [46]:
session_service = InMemorySessionService()  # Handles conversations

# Create a runner for the auto-save agent
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
    plugins=[
        LoggingPlugin()
    ]  # <---- Add the plugin. Handles standard Observability logging across ALL agents
)

print("✅ Runner created.")

✅ Runner created.


In [47]:
await run_session(
    auto_runner,
    "I want to apply for Senior Data Engineer position at Goto company. Can you give me details on the company.",
    "session_1",
)


### Session: session_1

User > I want to apply for Senior Data Engineer position at Goto company. Can you give me details on the company.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-124373b5-648b-451a-a51f-828a1a8207be
[logging_plugin]    Session ID: session_1
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'I want to apply for Senior Data Engineer position at Goto company. Can you give me details on the company.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-124373b5-648b-451a-a51f-828a1a8207be
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-124373b5-648b-451a-a51f-828a1a8207be
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: Helpi

In [29]:
await run_session(
    auto_runner,
    "Yes, want to know about the GoTo Technologies USA, Inc. (formerly LogMeIn)",
    "session_1",
)


### Session: session_1

User > Yes, want to know about the GoTo Technologies USA, Inc. (formerly LogMeIn)
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-7cdbed41-947b-462c-9cb3-97af21a159fa
[logging_plugin]    Session ID: session_1
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'Yes, want to know about the GoTo Technologies USA, Inc. (formerly LogMeIn)'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-7cdbed41-947b-462c-9cb3-97af21a159fa
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-7cdbed41-947b-462c-9cb3-97af21a159fa
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: HelpingAgent
[logging_plugin]    System Instruction: 'You are a Helpe

In [48]:
await run_session(
    auto_runner,
    "Ok Thanks! Looks good to me! I have applied for the role of `Senior Data Engineer` at `GoTo Technologies USA, Inc. (formerly LogMeIn)` on Nov 21.",
    "session_1",
)


### Session: session_1

User > Ok Thanks! Looks good to me! I have applied for the role of `Senior Data Engineer` at `GoTo Technologies USA, Inc. (formerly LogMeIn)` on Nov 21.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e4590b78-a295-4cb6-8e5f-7866dfb487b3
[logging_plugin]    Session ID: session_1
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'Ok Thanks! Looks good to me! I have applied for the role of `Senior Data Engineer` at `GoTo Technologies USA, Inc. (formerly LogMeIn)` on Nov 21.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-e4590b78-a295-4cb6-8e5f-7866dfb487b3
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-e4590b78-a295-4cb6-8e5f-7866dfb487b3
[logging_plugin] 🧠 LLM REQUEST
[lo

In [49]:
await run_session(
    auto_runner,
    """Can you tally with me all the jobs I have applied so far and what's the latest status know to you.""",
    "session_2",
)


### Session: session_2

User > Can you tally with me all the jobs I have applied so far and what's the latest status know to you.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-ea741091-ccde-46b3-a515-fe8652382694
[logging_plugin]    Session ID: session_2
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'Can you tally with me all the jobs I have applied so far and what's the latest status know to you.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-ea741091-ccde-46b3-a515-fe8652382694
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-ea741091-ccde-46b3-a515-fe8652382694
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: HelpingAgent
[logging

In [43]:
await run_session(
    auto_runner,
    """now, Can you help list all the labels in my gmail account.""",
    "session_3",
)


### Session: session_3

User > now, Can you help list all the labels in my gmail account.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e2253532-c120-4c3e-8291-19f66862f352
[logging_plugin]    Session ID: session_3
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'now, Can you help list all the labels in my gmail account.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-e2253532-c120-4c3e-8291-19f66862f352
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-e2253532-c120-4c3e-8291-19f66862f352
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: HelpingAgent
[logging_plugin]    System Instruction: 'You are a Helper agent for the user. User can g

In [50]:
await run_session(
    auto_runner,
    """now, Can you help summarize last 2 days of my emails for my gmail account 'npatel17da@gmail.com' under `UPDATES` label""",
    "session_3",
)


### Session: session_3

User > now, Can you help summarize last 2 days of my emails for my gmail account 'npatel17da@gmail.com' under `UPDATES` label
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-df5b4f0c-84a3-471d-8755-2723ec198853
[logging_plugin]    Session ID: session_3
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'now, Can you help summarize last 2 days of my emails for my gmail account 'npatel17da@gmail.com' under `UPDATES` label'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-df5b4f0c-84a3-471d-8755-2723ec198853
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: HelpingAgent
[logging_plugin]    Invocation ID: e-df5b4f0c-84a3-471d-8755-2723ec198853
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging

d:\code_folder\.myvenv\Lib\site-packages\google\adk\tools\mcp_tool\mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: gmail_search_agent
[logging_plugin]    System Instruction: 'Use the `get_gmail_labels` tool to get list of available labels in the user's gamil account. Please give as input users' email address to this tool.
    If you don't have email address please ask the ...'
[logging_plugin]    Available Tools: ['get_gmail_labels', 'get_emails']
[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: gmail_search_agent
[logging_plugin]    Content: function_call: get_emails
[logging_plugin]    Token Usage - Input: 598, Output: 41
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 61784997-b6e7-4046-b005-61523bec68a0
[logging_plugin]    Author: gmail_search_agent
[logging_plugin]    Content: function_call: get_emails
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['get_emails']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: get_ema

In [ ]:
await run_session(
    auto_runner,
    """Can you please go through my emails under INBOX and UPDATES labels to list out all the jobs I have applied ,
    and where I have recived confirmation from the company of them receiving my application in last 2 days for the email id `npatel17da@gmail.com`""",
    "session_4",
)


### Session: session_4

User > Can you please go through my emails under INBOX and UPDATES labels to list out all the jobs I have applied ,
    and where I have recived confirmation from the company of them receiving my application in last 7 days for the email id `npatel17da@gmail.com`
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-975a87c3-faa4-45da-be73-329929409169
[logging_plugin]    Session ID: session_4
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: MemoryDemoApp
[logging_plugin]    Root Agent: HelpingAgent
[logging_plugin]    User Content: text: 'Can you please go through my emails under INBOX and UPDATES labels to list out all the jobs I have applied ,
    and where I have recived confirmation from the company of them receiving my application...'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-975a87c3-faa4-45da-be73-329929409169
[logging_plugin]    Starting Agent: HelpingAgent
[logging_plugin] 🤖 

d:\code_folder\.myvenv\Lib\site-packages\google\adk\tools\mcp_tool\mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: gmail_search_agent
[logging_plugin]    System Instruction: 'Use the `get_gmail_labels` tool to get list of available labels in the user's gamil account. Please give as input users' email address to this tool.
    If you don't have email address please ask the ...'
[logging_plugin]    Available Tools: ['get_gmail_labels', 'get_emails']
[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: gmail_search_agent
[logging_plugin]    Content: text: 'I cannot filter emails based on confirmation of receiving application. I can only filter based on labels and date.'
[logging_plugin]    Token Usage - Input: 604, Output: 21
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 2a2dd545-991e-4c10-966c-ce8d02baf33c
[logging_plugin]    Author: gmail_search_agent
[logging_plugin]    Content: text: 'I cannot filter emails based on confirmation of receiving application. I can only filt

In [ ]:
await run_session(
    auto_runner,
    """Sure, that works... """,
    "session_4",
)
# Limit exhausted for the test gmail credentials